In [1]:
import pickle
import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from predictive_model import le # Import label encoder

class FeedbackSystem:
    def __init__(self):
        self.model = None
        self.pca = None
        self.data = []
        self.labels = []
        self.load_initial_model()

    def load_initial_model(self):
        # Load pre-trained PCA and KMeans
        self.pca = PCA(n_components=35, whiten=True, svd_solver='full')
        self.pca.fit(pickle.load(open('../data/features/VGG16_fc1_features_std.pickle', 'rb'))['features'])
        self.model = KMeans(n_clusters=7, init='k-means++', n_init=500)
        self.model.fit(self.pca.transform(fc1))  # Assuming `fc1` is loaded globally

    def process_feedback(self, image_features):
        # Predict
        reduced_features = self.pca.transform([image_features])
        cluster = self.model.predict(reduced_features)[0]
        predicted_label = le.inverse_transform([cluster])[0]
        
        # Get feedback
        is_correct = input(f"Predicted: {predicted_label}. Correct? (y/n): ").lower() == 'y'
        
        if not is_correct:
            is_new_class = input("Is this a new class? (y/n): ").lower() == 'y'
            if is_new_class:
                new_class = input("Enter class name: ")
                le.classes_ = np.append(le.classes_, new_class)
                print(f"New class '{new_class}' added.")
            else:
                correct_label = input("Enter correct label: ")
                print("Model update requires retraining. Use retrain_model() later.")
        else:
            print("Feedback recorded. Thank you!")
        
    def retrain_model(self, new_data, new_labels):
        # Append new data and retrain
        self.data.extend(new_data)
        self.labels.extend(new_labels)
        updated_features = self.pca.transform(self.data)
        self.model = KMeans(n_clusters=len(le.classes_), init='k-means++', n_init=100)
        self.model.fit(updated_features)
        print("Model retrained with new data.")

In [ ]:
# Example usage
fb_system = FeedbackSystem()
new_image_features = [...]  # Extract features for a new image
fb_system.process_feedback(new_image_features)
# After collecting enough feedback data:
# fb_system.retrain_model(new_data_list, new_label_list)